`Dataframe` to `DocumentArray`...

In [28]:
from docarray import Document, DocumentArray
import pandas

from k_fold import data_selection

train_df, test_df = data_selection('C:/Users/lachy/Pictures/utkface/utkface/')

train_da = DocumentArray.from_dataframe(train_df)
test_da = DocumentArray.from_dataframe(test_df)

# for item in train_da:
#     item.summary()


c:\Users\lachy\Videos\LACHY\SRI Repo\bias-project-ML\debiased_clip\k_fold.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][i]= gender_mapper[df['gender'][i]]
c:\Users\lachy\Videos\LACHY\SRI Repo\bias-project-ML\debiased_clip\k_fold.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'][i]= race_mapper[df['race'][i]]


Initializing `CLIP`...

In [29]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

`CLIP` Fine-tuning...

In [30]:
import finetuner

finetuner.login()

`Race ` and ` Gender` Fine-tuning...

In [45]:
gender_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['gender'] + ' gender and ' + doc.tags['race'] + ' race.' )
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    gender_pairs.append(pair)

# Lets see the first item of the pairs
gender_pairs[0]

📄 Document: dd860365b746fb3269cb57857b9ef3ea
└── 💠 Chunks
    ├── 📄 Document: 82a75c086d0aa1ab437ceeb0fded922c
    │   ╭───────────┬──────────────────────────────────────────────────────────────────╮
    │   │ Attribute │ Value                                                            │
    │   ├───────────┼──────────────────────────────────────────────────────────────────┤
    │   │ tensor    │ <class 'numpy.ndarray'> in shape (224, 224, 3), dtype: uint8     │
    │   │ mime_type │ image/jpeg                                                       │
    │   │ uri       │ C:/Users/lachy/Pictures/utkface/utkface/100_0_0_201701122135009… │
    │   │ tags      │ {'filename': '100_0_0_20170112213500903.jpg.chip.jpg', 'gender': │
    │   │           │ 'male', 'race': 'white', 'filepath':                             │
    │   │           │ 'C:/Users/lachy/Pictures/utkface/utkface/100_0_0_20170112213500… │
    │   │           │ 'age': 100}                                                      │
    │   │ modality  │ image                                                            │
    │   ╰───────────┴──────────────────────────────────────────────────────────────────╯
    └── 📄 Document: 531560819869caed81c3b2ec93010bbe
        ╭───────────────┬──────────────────────────────────────────────────────────────╮
        │ Attribute     │ Value                                                        │
        ├───────────────┼──────────────────────────────────────────────────────────────┤
        │ text          │ This is a person of male gender and white race.              │
        │ modality      │ text                                                         │
        ╰───────────────┴──────────────────────────────────────────────────────────────╯

In [44]:
len(gender_pairs)

18966

Training `CLIP` model...

In [46]:
run = finetuner.fit(
    model='openai/clip-vit-base-patch32', # fine-tune CLIP
    train_data=gender_pairs,   
    learning_rate=1e-5,
    loss='CLIPLoss'
)

Pushing a DocumentArray to Hubble under the name finetuner-dastorage-default-elated-easley-train ...


Output()

In [ ]:
for entry in run.stream_logs():
    print(entry)

Saving the artifact

In [ ]:
artifact = run.save_artifact('/artifact/')

In [ ]:
clip_text_encoder = finetuner.get_model(artifact=artifact, select_model='clip-text')
clip_image_encoder = finetuner.get_model(artifact=artifact, select_model='clip-vision')